<a href="https://colab.research.google.com/github/teerasitk/EEMATH/blob/main/ErrorCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [161]:
import numpy as np
import matplotlib.pyplot as plt

In [162]:
def compSyndrone(decision:np.ndarray, H:np.ndarray)->np.ndarray:
  """
  decision: 7xn binary array
  H: hamming decoder matrix of size 7 x 3
  return syndrone binary array of size 3 x N
  """
  s = (np.dot(H.T, decision) % 2).astype('int')
  return s

# error_pattern = [[0, 0, 0, 0, 0, 0, 0, 0], # 000
#                  [0, 0, 0, 0, 0, 0, 0, 1], # 001
#                  [0, 0, 0, 0, 0, 0, 1, 0], # 010
#                  [0, 0, 1, 0, 0, 0, 0, 0], # 011
#                  [0, 0, 0, 0, 0, 1, 0, 0], # 100
#                  [0, 1, 0, 0, 0, 0, 0, 0], # 101
#                  [1, 0, 0, 0, 0, 0, 0, 0], # 110
#                  [0, 0, 0, 1, 0, 0, 0, 0]] # 111

def errorCorrection(decision, H, error_pattern):
  syndrome = compSyndrone(decision, H).astype('int')
  value = syndrome[0, :] *4 + syndrome[1, :] * 2  + syndrome[2, :]
  error_loc = error_pattern[value].T
  corrected_decision = (decision + error_loc) %2
  return corrected_decision

In [150]:
def hammingEncoder(src: np.ndarray) -> np.ndarray:
  """
  src: 4 x N binary array
  return: 7 x n binary array
  """
  A = np.array([[1, 1, 0],
                [1, 0, 1],
                [0, 1, 1],
                [1, 1, 1]])
  I4 = np.eye(4)
  G = np.concatenate(([I4, A.T]), axis=0) # Encoder Matrix
  hamming_src = np.dot(G, src) % 2 # modular 2 to make sure that the output is binary
  return hamming_src.astype('int')


def hammingDecoder(src: np.ndarray) -> np.ndarray:
  """
  src: 7 x N binary array
  return: 4 x n binary array
  """
  A = np.array([[1, 1, 0],
              [1, 0, 1],
              [0, 1, 1],
              [1, 1, 1]])
  I3 = np.eye(3)
  H = np.concatenate(([A, I3]), axis=0) # Decoder Matrinx
  error_pattern = [np.zeros(7,) for k in range(8)] # extract Syndrome
  a = np.zeros((7,)) # no error first
  s = np.dot(H.T,a).astype('int')
  error_pattern[s[0]*4 + s[1] *2 + s[2]] = a
  for k in range(7):
    a = np.zeros((7,))
    a[k] = 1
    s = np.dot(H.T,a).astype('int')
    error_pattern[s[0] * 4 + s[1] * 2 + s[2]] = a
  error_pattern = np.array(error_pattern)
  corrected_codes =  errorCorrection(src, H, error_pattern)
  return corrected_codes[:4, :].astype('int')


In [169]:
# parameter for experimental setup
N = 8 # number samples
noise_magnitude  = 1.0 # noise magnitude
SNR =  1.0/(noise_magnitude ** 2)
SNR_dB = 10 * np.log10(SNR)
print(f"Signal to noise ration: {SNR_dB:0.2f} dB")

Signal to noise ration: 0.00 dB


In [170]:
# source encoder
N = 8 # used only 8 samples
src = np.random.choice(2, size=(4, N)) # 8 binary codes of size 4 bits
hamming_src = hammingEncoder(src)
for k in range(N):
  print("Input Binary 4 bits: ", src[:, k], "Hamming Code 7 bits: ", hamming_src[:, k])


Input Binary 4 bits:  [1 0 1 0] Hamming Code 7 bits:  [1 0 1 0 1 0 1]
Input Binary 4 bits:  [1 0 0 1] Hamming Code 7 bits:  [1 0 0 1 0 0 1]
Input Binary 4 bits:  [0 0 1 1] Hamming Code 7 bits:  [0 0 1 1 1 0 0]
Input Binary 4 bits:  [0 0 1 0] Hamming Code 7 bits:  [0 0 1 0 0 1 1]
Input Binary 4 bits:  [1 1 1 1] Hamming Code 7 bits:  [1 1 1 1 1 1 1]
Input Binary 4 bits:  [0 0 0 1] Hamming Code 7 bits:  [0 0 0 1 1 1 1]
Input Binary 4 bits:  [1 1 0 1] Hamming Code 7 bits:  [1 1 0 1 1 0 0]
Input Binary 4 bits:  [1 1 0 0] Hamming Code 7 bits:  [1 1 0 0 0 1 1]


In [171]:
# convert src into signal to send "1" we use 1 volt to send "0" we use -1 volt.
signal = hamming_src  * 2 - 1
for k in range(N):
  print("Hamming Code 7 bits: ", hamming_src[:, k], "Signal : ", signal[:, k])

Hamming Code 7 bits:  [1 0 1 0 1 0 1] Signal :  [ 1 -1  1 -1  1 -1  1]
Hamming Code 7 bits:  [1 0 0 1 0 0 1] Signal :  [ 1 -1 -1  1 -1 -1  1]
Hamming Code 7 bits:  [0 0 1 1 1 0 0] Signal :  [-1 -1  1  1  1 -1 -1]
Hamming Code 7 bits:  [0 0 1 0 0 1 1] Signal :  [-1 -1  1 -1 -1  1  1]
Hamming Code 7 bits:  [1 1 1 1 1 1 1] Signal :  [1 1 1 1 1 1 1]
Hamming Code 7 bits:  [0 0 0 1 1 1 1] Signal :  [-1 -1 -1  1  1  1  1]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] Signal :  [ 1  1 -1  1  1 -1 -1]
Hamming Code 7 bits:  [1 1 0 0 0 1 1] Signal :  [ 1  1 -1 -1 -1  1  1]


In [172]:
# All transmission in communication system is noisy,
noise_magnitude  = 1.0
received_signal = signal + noise_magnitude * np.random.randn(7, 8) # 7 bits 8 codes
for k in range(N):
  print("Signal : ", signal[:, k], "Recieved Signal: ", received_signal[:, k] )

Signal :  [ 1 -1  1 -1  1 -1  1] Recieved Signal:  [ 0.37906462 -1.36121899  1.37518951 -0.42057728  0.31480979 -0.19247272
  1.0640983 ]
Signal :  [ 1 -1 -1  1 -1 -1  1] Recieved Signal:  [ 0.15408287 -0.27334931 -0.73079165  0.37794305 -0.70128997 -0.33513777
 -0.41189858]
Signal :  [-1 -1  1  1  1 -1 -1] Recieved Signal:  [-2.64686666 -1.85095154  1.40505591  1.28139867 -0.00401405 -0.94724868
 -1.26949399]
Signal :  [-1 -1  1 -1 -1  1  1] Recieved Signal:  [-1.25679134 -1.43223535  0.31389933 -1.67270011 -1.11007211  1.05469362
  1.06126004]
Signal :  [1 1 1 1 1 1 1] Recieved Signal:  [-1.60658745e-03  9.72135055e-01  4.46914599e-01  1.18785500e-01
  6.03453875e-01  2.57793786e+00  1.03284628e+00]
Signal :  [-1 -1 -1  1  1  1  1] Recieved Signal:  [-3.04689617 -2.02136631 -0.26115422  1.85854177  0.51959166  0.81359729
  2.18606938]
Signal :  [ 1  1 -1  1  1 -1 -1] Recieved Signal:  [ 1.49998847 -0.5060233  -2.33604368  2.98244678  1.78462928 -1.15566817
 -1.32823711]
Signal :  [ 1

In [173]:
# make binary decision of recieved signal >= 0, we decide "1" and if signal <0, we decide "1"
decided_code = (received_signal >=0).astype('int')
for k in range(N):
  print("Hamming Code 7 bits: ", hamming_src[:, k], "decided code: ", decided_code[:, k] )
num_bit_errors = (hamming_src != decided_code).sum()
error_rate = (hamming_src != decided_code).mean()
print(f"There are {num_bit_errors} from {N*7} bits with error rate {error_rate*100: 0.3f}%")
# 4 bit data
num_bit_errors = (hamming_src[:4, :] != decided_code[:4, :]).sum()
error_rate = (hamming_src[:4, :] != decided_code[:4, :]).mean()
print(f"For the first 4 bit data. There are {num_bit_errors} from {N*4} bits with error rate {error_rate*100: 0.3f}%")

Hamming Code 7 bits:  [1 0 1 0 1 0 1] decided code:  [1 0 1 0 1 0 1]
Hamming Code 7 bits:  [1 0 0 1 0 0 1] decided code:  [1 0 0 1 0 0 0]
Hamming Code 7 bits:  [0 0 1 1 1 0 0] decided code:  [0 0 1 1 0 0 0]
Hamming Code 7 bits:  [0 0 1 0 0 1 1] decided code:  [0 0 1 0 0 1 1]
Hamming Code 7 bits:  [1 1 1 1 1 1 1] decided code:  [0 1 1 1 1 1 1]
Hamming Code 7 bits:  [0 0 0 1 1 1 1] decided code:  [0 0 0 1 1 1 1]
Hamming Code 7 bits:  [1 1 0 1 1 0 0] decided code:  [1 0 0 1 1 0 0]
Hamming Code 7 bits:  [1 1 0 0 0 1 1] decided code:  [1 1 0 0 0 1 1]
There are 4 from 56 bits with error rate  7.143%
For the first 4 bit data. There are 2 from 32 bits with error rate  6.250%


In [174]:
# Using Hamming decoder
hamming_decoded_code = hammingDecoder(decided_code)
for k in range(N):
  print("source Code 4 bits: ", src[:, k], "hamming decoded code: ", hamming_decoded_code[:, k] )
num_bit_errors = (src !=hamming_decoded_code).sum()
error_rate = (src != hamming_decoded_code).mean()
print(f"For the first 4 bit data with Hamming Decoder.")
print(f"There are {num_bit_errors} from {N*4} bits with error rate {error_rate*100: 0.3f}%")

source Code 4 bits:  [1 0 1 0] hamming decoded code:  [1 0 1 0]
source Code 4 bits:  [1 0 0 1] hamming decoded code:  [1 0 0 1]
source Code 4 bits:  [0 0 1 1] hamming decoded code:  [0 0 1 1]
source Code 4 bits:  [0 0 1 0] hamming decoded code:  [0 0 1 0]
source Code 4 bits:  [1 1 1 1] hamming decoded code:  [1 1 1 1]
source Code 4 bits:  [0 0 0 1] hamming decoded code:  [0 0 0 1]
source Code 4 bits:  [1 1 0 1] hamming decoded code:  [1 1 0 1]
source Code 4 bits:  [1 1 0 0] hamming decoded code:  [1 1 0 0]
For the first 4 bit data with Hamming Decoder.
There are 0 from 32 bits with error rate  0.000%
